# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [4]:
data.shape

(68584, 7)

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [5]:
s1 = data[['Quantity', 'CustomerID', 'ProductName']].groupby(['CustomerID', 'ProductName'], as_index=False).agg('sum')

In [7]:
s1.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [9]:
s1.shape

(63628, 3)

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [12]:
s2 = pd.pivot_table(data, 
                    values='Quantity', 
                    columns='ProductName', 
                    index='CustomerID', 
                    aggfunc='sum', fill_value= 0.0)
s2.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [13]:
from scipy.spatial.distance import pdist, squareform
# SQUAREFORM sirve para que mi output me lo dé en forma de matriz
squareform(pdist(s2, 'euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [14]:
distances = pd.DataFrame(1/(1 + squareform(pdist(s2, 'euclidean'))), 
                         index=s2.index, columns=s2.index)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [29]:
similarities = distances[412].sort_values(ascending=False)[1:6]
similarities

CustomerID
3317    0.085638
3535    0.084959
1008    0.083333
1072    0.082709
639     0.082403
Name: 412, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
s5 = s1.loc[s1.CustomerID.isin(list(similarities.index))]
s5.head()

,CustomerID,ProductName,Quantity
452,639,Anchovy Paste - 56 G Tube,1
453,639,Appetizer - Mushroom Tart,1
454,639,Assorted Desserts,1
455,639,Bread - Italian Roll With Herbs,1
456,639,Bread - Raisin Walnut Oval,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [18]:
s6 = s5[['ProductName', 'Quantity']].groupby('ProductName', as_index=False).agg('sum').sort_values(by='Quantity', ascending=False).reset_index()
s6.head()

,index,ProductName,Quantity
0,123,Olive - Spread Tapenade,3
1,175,Sprouts - Baby Pea Tendrils,3
2,201,Wine - Blue Nun Qualitatswein,3
3,193,"Veal - Inside, Choice",3
4,134,"Pepper - Black, Whole",3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [19]:
# List of already bought products
bought = list(data.ProductName.loc[(data.CustomerID == 412)])

# Filtering the dataframe by products that are not in the list
s7 = s6.loc[~s6.ProductName.isin(bought)]
s7.head()

,index,ProductName,Quantity
0,123,Olive - Spread Tapenade,3
1,175,Sprouts - Baby Pea Tendrils,3
2,201,Wine - Blue Nun Qualitatswein,3
4,134,"Pepper - Black, Whole",3
5,168,Soup - Campbells Bean Medley,3


In [36]:
list(s7.ProductName.head())

['Scallops - Live In Shell',
 'Chicken - Soup Base',
 'Veal - Inside',
 'Pepper - Paprika, Hungarian',
 'Puree - Mocha']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [37]:
recommendations = {}
customers = list(data.CustomerID.value_counts().index)
for i in customers:
        similarities = distances[i].sort_values(ascending=False)[1:6]
        s5 = s1.loc[s1.CustomerID.isin(list(similarities.index))]
        s6 = s5[['ProductName', 'Quantity']].groupby('ProductName', as_index=False).agg('sum').sort_values(by='Quantity', ascending=False)
        bought = list(data.ProductName.loc[(data.CustomerID == i)])
        s7 = s6.loc[~s6.ProductName.isin(bought)]
        recommendations[i] = list(s7.ProductName.head())

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [39]:
item_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
item_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
item_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33759,"Coconut - Shredded, Sweet",Shrimp - 31/40,Lambcasing,Bread - French Baquette,Pecan Raisin - Tarts
1,60862,Onions - Vidalia,Chips Potato Salt Vinegar 43g,"Pepsi - Diet, 355 Ml",Sugar - Fine,Wine - Valpolicella Masi
2,29287,Beer - Sleemans Cream Ale,Bread - Italian Corn Meal Poly,Table Cloth 54x72 White,Cheese - Wine,Cookie - Dough Variety
3,8711,Sauce - Hollandaise,Soup - Campbells Tomato Ravioli,Dried Figs,Salmon - Sockeye Raw,Veal - Eye Of Round
4,63086,Sausage - Liver,Beef - Top Sirloin - Aaa,Oil - Shortening - All - Purpose,Sardines,"Wine - Red, Colio Cabernet"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

#### Cosine

In [40]:
distances = pd.DataFrame(1/(1 + squareform(pdist(s2, 'cosine'))), 
                         index=s2.index, columns=s2.index)

In [41]:
recommendations = {}
customers = list(data.CustomerID.value_counts().index)
for i in customers:
        similarities = distances[i].sort_values(ascending=False)[1:6]
        s5 = s1.loc[s1.CustomerID.isin(list(similarities.index))]
        s6 = s5[['ProductName', 'Quantity']].groupby('ProductName', as_index=False).agg('sum').sort_values(by='Quantity', ascending=False)
        bought = list(data.ProductName.loc[(data.CustomerID == i)])
        s7 = s6.loc[~s6.ProductName.isin(bought)]
        recommendations[i] = list(s7.ProductName.head())

In [42]:
item_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
item_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
item_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33759,Truffle Cups - Brown,Flavouring - Orange,Tia Maria,Beans - Wax,Beef - Texas Style Burger
1,60862,Snapple Lemon Tea,"Coconut - Shredded, Sweet",Ketchup - Tomato,Rabbit - Whole,Foam Cup 6 Oz
2,29287,Dc Hikiage Hira Huba,"Beans - Kidney, Canned",Oil - Shortening - All - Purpose,Assorted Desserts,"Chestnuts - Whole,canned"
3,8711,Veal - Sweetbread,Wine - Redchard Merritt,Yogurt - French Vanilla,Cream Of Tartar,Olive - Spread Tapenade
4,63086,Pecan Raisin - Tarts,Mangoes,Truffle Cups - Brown,Crackers - Trio,Tea - Jasmin Green


#### Manhattan 

In [45]:
distances = pd.DataFrame(1/(1 + squareform(pdist(s2, 'cityblock'))), 
                         index=s2.index, columns=s2.index)

In [46]:
recommendations = {}
customers = list(data.CustomerID.value_counts().index)
for i in customers:
        similarities = distances[i].sort_values(ascending=False)[1:6]
        s5 = s1.loc[s1.CustomerID.isin(list(similarities.index))]
        s6 = s5[['ProductName', 'Quantity']].groupby('ProductName', as_index=False).agg('sum').sort_values(by='Quantity', ascending=False)
        bought = list(data.ProductName.loc[(data.CustomerID == i)])
        s7 = s6.loc[~s6.ProductName.isin(bought)]
        recommendations[i] = list(s7.ProductName.head())

In [47]:
item_recs = pd.DataFrame.from_dict(recommendations, orient='index').reset_index()
item_recs.columns = ['CustomerID', 'Rec1', 'Rec2', 'Rec3', 'Rec4', 'Rec5']
item_recs.head()

,CustomerID,Rec1,Rec2,Rec3,Rec4,Rec5
0,33759,Muffin - Carrot Individual Wrap,"Cheese - Brie, Triple Creme",Soup - Campbells Tomato Ravioli,Halibut - Steaks,Ice Cream Bar - Hageen Daz To
1,60862,"Pork - Bacon, Double Smoked","Nut - Chestnuts, Whole",Butter - Unsalted,Soup - Campbells Bean Medley,"Salsify, Organic"
2,29287,Table Cloth 54x72 White,"Salsify, Organic",Foam Dinner Plate,V8 - Berry Blend,Milk Powder
3,8711,Pork - Kidney,"Wine - White, Colubia Cresh",Sauce - Hollandaise,Veal - Eye Of Round,Soup - Campbells Tomato Ravioli
4,63086,"Bar Mix - Pina Colada, 355 Ml",Ice Cream Bar - Hageen Daz To,Black Currants,Beer - Original Organic Lager,"Spoon - Soup, Plastic"
